In [1]:
import numpy as np
import pandas as pd

In [256]:
df = {}
year_start, year_end = 2003,2020
years_iter = range(year_start, year_end)

skiprows_dict = { 2003:1, 2004:1, 2005:2, 2006:2, 2007:2, 2008:2, 2009:2, 
             2010:2, 2011:2, 2012:2, 2013:3, 2014:3, 2015:3, 2016:3, 2017:3, 2018:3, 2019:3  }

columns = ['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound.1', '90% CI Upper Bound.1',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound.2',
       '90% CI Upper Bound.2', 'Poverty Percent, Age 0-17',
       '90% CI Lower Bound.3', '90% CI Upper Bound.3',
       'Poverty Estimate, Age 5-17 in Families', '90% CI Lower Bound.4',
       '90% CI Upper Bound.4', 'Poverty Percent, Age 5-17 in Families',
       '90% CI Lower Bound.5', '90% CI Upper Bound.5',
       'Median Household Income', '90% CI Lower Bound.6',
       '90% CI Upper Bound.6', 'Poverty Estimate, Age 0-4',
       '90% CI Lower Bound.7', '90% CI Upper Bound.7',
       'Poverty Percent, Age 0-4', '90% CI Lower Bound.8',
       '90% CI Upper Bound.8']

for year in years_iter:
    df[year] = pd.read_excel('data/saipe/est'+'{:02}'.format(year%100)+'all.xls',skiprows=skiprows_dict[year], names=columns)
    #print( df[year].columns[-1])

In [268]:
FIPS_county = df[2003][ ['State FIPS Code', 'County FIPS Code','Postal Code','Name'] ].drop_duplicates()
FIPS_county_dict = { (FIPS_county['State FIPS Code'].iloc[i],FIPS_county['County FIPS Code'].iloc[i]):
                     (FIPS_county['Postal Code'].iloc[i],FIPS_county['Name'].iloc[i]) for i in range( len(FIPS_county) )
                   }

def povinc_county(row):
    splitted = row.values[0].split()
    
    FIPS = ( int(splitted[0]),int(splitted[1]) )
    if FIPS in FIPS_county_dict:
        postal = FIPS_county_dict[ FIPS ][0]
        county = FIPS_county_dict[ FIPS ][1]
        #poverty_perc = float(splitted[5])
        #income = float(splitted[20])
        poverty_perc = splitted[5]
        income = splitted[20]

        return [postal, county, poverty_perc , income] 
    else:
        return [ np.nan ]*4

In [269]:
for year in range(1997,2003):
    df_y = pd.read_table('data/saipe/est'+'{:02}'.format(year%100)+'all.dat',header=None)
    df[year] = pd.DataFrame( list( df_y.apply( povinc_county,axis=1) ) ,columns=['Postal Code','Name',
                                                                                 'Poverty Percent, All Ages','Median Household Income'] )
    df[year][ 'Poverty Percent, All Ages' ] = pd.to_numeric( df[year][ 'Poverty Percent, All Ages' ],errors='coerce' )
    df[year][ 'Median Household Income' ] = pd.to_numeric( df[year][ 'Median Household Income' ],errors='coerce' )
    df[year].dropna(inplace=True)

In [285]:
#df_1 = df[year_start][ ['Name', 'Poverty Estimate, All Ages','Poverty Percent, All Ages','Median Household Income'] ].copy()
#df_1 = {}
poverty_df = {}
income_df = {}

year_start, year_end = 1997,2020
years_iter = range(year_start, year_end)

for year in years_iter:
    df[year].drop_duplicates(subset=['Postal Code','Name'],inplace=True)
    #df_1[year] = df[year][ ['Postal Code', 'Name', 'Poverty Estimate, All Ages',
     #                       'Poverty Percent, All Ages','Median Household Income'] ].set_index( ['Postal Code','Name'] )
    poverty_df[year] = df[year][ ['Postal Code', 'Name', 'Poverty Percent, All Ages'] ].set_index( ['Postal Code','Name'] )
    poverty_df[year].rename( columns={'Poverty Percent, All Ages':year} ,inplace=True)
    
    income_df[year] = df[year][ ['Postal Code', 'Name', 'Median Household Income'] ].set_index( ['Postal Code','Name'] )
    income_df[year].rename( columns={'Median Household Income':year} ,inplace=True)
    

In [286]:

poverty_all = poverty_df[year_start]
income_all = income_df[year_start]

for year in range( year_start+1,year_end ):
    poverty_all = pd.merge(poverty_all, poverty_df[year],how='inner' ,on=['Postal Code','Name'],copy=False )
    #print( len(poverty_all) )
    #poverty_all.to_csv('data/draft/poverty_merged_'+str(year)+'_.csv')
    income_all = pd.merge(income_all, income_df[year],how='inner' ,on=['Postal Code','Name'],copy=False )
    
poverty_all = poverty_all.T
income_all = income_all.T

In [287]:
poverty_all.to_csv('data/saipe/poverty_percent_county_years_.csv')
income_all.to_csv('data/saipe/income_county_years_.csv')

# Draft

In [275]:
#poverty_df[2003]
df[2002]
#df_y

,Postal Code,Name,"Poverty Percent, All Ages",Median Household Income
0,US,United States,12.1,42409.0
1,AL,Alabama,15.4,35943.0
2,AL,Autauga County,10.3,42841.0
3,AL,Baldwin County,10.7,40818.0
4,AL,Barbour County,23.6,25279.0
...,...,...,...,...
3188,WY,Sweetwater County,7.9,50801.0
3189,WY,Teton County,5.6,57425.0
3190,WY,Uinta County,10.0,47821.0
3191,WY,Washakie County,10.8,38676.0


In [37]:
df_y = pd.read_excel('data/saipe/est18all.xls',skiprows=3)
df_y.columns

Index(['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound.1', '90% CI Upper Bound.1',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound.2',
       '90% CI Upper Bound.2', 'Poverty Percent, Age 0-17',
       '90% CI Lower Bound.3', '90% CI Upper Bound.3',
       'Poverty Estimate, Age 5-17 in Families', '90% CI Lower Bound.4',
       '90% CI Upper Bound.4', 'Poverty Percent, Age 5-17 in Families',
       '90% CI Lower Bound.5', '90% CI Upper Bound.5',
       'Median Household Income', '90% CI Lower Bound.6',
       '90% CI Upper Bound.6', 'Poverty Estimate, Age 0-4',
       '90% CI Lower Bound.7', '90% CI Upper Bound.7',
       'Poverty Percent, Age 0-4', '90% CI Lower Bound.8',
       '90% CI Upper Bound.8'],
      dtype='object')

In [103]:
df_y = pd.read_table('data/saipe/est02all.dat',header=None)  # ,sep='[\s]*',engine='python'
df_y

,0
0,00 0 34569951 33912173 35227729 12.1 11.9 12...
1,01 0 679856 646828 712885 15.4 14.7 16...
2,01 1 4795 3728 5863 10.3 8.0 12...
3,01 3 16175 12558 19793 10.7 8.3 13...
4,01 5 6152 4767 7538 23.6 18.3 28...
...,...
3188,56 37 2903 2262 3545 7.9 6.2 9...
3189,56 39 1037 777 1296 5.6 4.2 7...
3190,56 41 1951 1514 2388 10.0 7.8 12...
3191,56 43 836 644 1027 10.8 8.3 13...


In [117]:
#df_y.values[0][0].split()

In [251]:
def poverty_county(row):
    splitted = row.values[0].split()
    postal = FIPS_county_dict[ ( int(splitted[0]),int(splitted[1]) ) ][0]
    county = FIPS_county_dict[ ( int(splitted[0]),int(splitted[1]) ) ][1]
    poverty_perc = float(splitted[5])
    income = float(splitted[20])
    #return {'Postal':postal}
    return [postal, county, poverty_perc , income] 
    #return pd.DataFrame( [ [postal, county, poverty_perc] ] )
    
def split_rows(row):
    splitted = row.values[0].split()
    numbers = splitted[:23]
    postal = splitted[-3]
    if splitted[23].isnumeric():
        county = ' '.join( splitted[-5:-3] )
    else:
        county = ' '.join( splitted[23:-3] )
    #return len(splitted)
    return splitted[:8]
    return [postal]+[county]#+numbers

In [252]:
pd.DataFrame( list( df_y.iloc[:18].apply( poverty_county,axis=1) ) ,columns=['Postal Code','Name','Poverty Percent, All Ages','income'] )
#list(df_y.iloc[:18].apply( poverty_county,axis=1).values)

,Postal Code,Name,"Poverty Percent, All Ages",income
0,US,United States,12.1,42409.0
1,AL,Alabama,15.4,35943.0
2,AL,Autauga County,10.3,42841.0
3,AL,Baldwin County,10.7,40818.0
4,AL,Barbour County,23.6,25279.0
5,AL,Bibb County,17.7,31387.0
6,AL,Blount County,12.6,36386.0
7,AL,Bullock County,31.4,20137.0
8,AL,Butler County,20.4,25826.0
9,AL,Calhoun County,15.8,32013.0


In [134]:
df_y.iloc[:7].apply( lambda x: x.values[0].split() ,axis=0)

,0
0,00
1,0
2,34569951
3,33912173
4,35227729
5,12.1
6,11.9
7,12.3
8,12132645
9,11795343


In [142]:
df_y.values[3]

array(['01   3    16175    12558    19793 10.7  8.3 13.1     5501     4146     6857 15.5 11.7 19.4     3735     2696     4774 14.3 10.3 18.3  40818  38197  43620                                        Baldwin County                                AL est02ALL.dat 29OCT2004'],
      dtype=object)

In [193]:
df[2003]

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound.1,90% CI Upper Bound.1,...,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,"Poverty Estimate, Age 0-4",90% CI Lower Bound.7,90% CI Upper Bound.7,"Poverty Percent, Age 0-4",90% CI Lower Bound.8,90% CI Upper Bound.8
0,0,0,US,United States,35861170,35190401,36531939,12.5,12.2,12.7,...,16.6,43318,43009,43627,4021366.0,3817517.0,4225215.0,20.3,19.3,21.4
1,1,0,AL,Alabama,677455,644647,710263,15.3,14.5,16,...,22.5,36131,34659,37602,72477.0,62361.0,82594.0,24.7,21.3,28.2
2,1,1,AL,Autauga County,4926,3815,6038,10.4,8.1,12.7,...,18.4,44241,41547,47109,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,AL,Baldwin County,15753,12234,19273,10.1,7.9,12.4,...,18,41961,39479,44600,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,AL,Barbour County,5717,4411,7022,22.2,17.1,27.3,...,37.9,26062,24432,27801,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56,37,WY,Sweetwater County,3192,2491,3894,8.6,6.7,10.5,...,11.1,52541,49470,55803,NaN,NaN,NaN,NaN,NaN,NaN
3189,56,39,WY,Teton County,1067,799,1335,5.6,4.2,7.1,...,8.6,57485,53397,61886,NaN,NaN,NaN,NaN,NaN,NaN
3190,56,41,WY,Uinta County,2063,1601,2525,10.6,8.2,12.9,...,12.2,48658,45744,51757,NaN,NaN,NaN,NaN,NaN,NaN
3191,56,43,WY,Washakie County,860,664,1055,11.0,8.5,13.5,...,17.7,40357,37887,42988,NaN,NaN,NaN,NaN,NaN,NaN
